# Layers

Layer in torch is some transformation with some inputs and some outputs.

In [1]:
import torch
from torch import nn

To perform a layer transformation on the tensor `X`, simply use the syntax `layer(X)`.

## Linear

The `torch.nn.Linear` layer performs the following operation:

$$X_{n \times l} \cdot \omega_{l \times k} + b_k$$

Where:

- $l$ - number of inputs
- $k$ - number of outputs
- $n$ - number of input samples
- $X_{n \times l}$ - input tensor
- $\omega_{l \times k}$ - weight matrix of the layer
- $b_k$ - bias vector of the layer

---

The following cell applies the tensor to some data and manually performs the same transformation. The results should be identical.

In [2]:
in_features = 5
out_features = 3

linear = nn.Linear(
    in_features = in_features, 
    out_features = out_features
)

X = torch.rand(in_features)

print("Layer transformation")
print(linear(X).tolist())
print("X@w+b")
print((linear.weight@X + linear.bias).tolist())

Layer transformation
[-0.6235317587852478, -1.1335619688034058, -0.70122230052948]
X@w+b
[-0.6235317587852478, -1.1335619688034058, -0.70122230052948]


### Define values

To define custom values for tensors you have to use access `weight` and `bias` fater layer creation. They are belongs to the `type(linear_layer.weight)` data type. So you have to use method `copy_` under `torch.no_grad` context.

---

Here’s an example of how to do it:

In [3]:
linear_layer = nn.Linear(in_features=3, out_features=4)

default_weights = torch.ones_like(linear_layer.weight)
default_biases = torch.zeros_like(linear_layer.bias)

with torch.no_grad():
    linear_layer.weight.copy_(default_weights)
    linear_layer.bias.copy_(default_biases)

After completing the process, we have the `weight` tensor initialized with ones and the `bias` tensor initialized with zeros:

In [4]:
print(linear_layer.weight)
print(linear_layer.bias)

Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)
